<a href="https://colab.research.google.com/github/fmbento/Jupyter-Notebooks_ad-hoc-solutions/blob/main/Gera_Acordi%C3%A3o_das_FAQs_do_LibGuides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ... basta correr a célula abaixo e de seguida (para as perguntas de um determinado grupo):
1.   Inserir a pergunta e a respectiva resposta (pode ser código HTML);
2.   Clicar em  "Adicionar Par P&R";
3.   Repetir os passos 1. e 2. para todas as perguntas do grupo;
4.   Adicionados todos os pares P&R do grupo, clicar em "Concluir";
5.   Copiar o código gerado e colar na fonte da box do LibGuides;
6.   Para fazer o próximo grupo de perguntas, executar a célula novamente.

In [ ]:
# @title Executar esta célula:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import uuid

class LibGuidesAccordionBuilder:
    def __init__(self):
        self.qa_pairs = []
        self.unique_id = str(uuid.uuid4())[:8]

        # Create widgets
        self.question_input = widgets.Textarea(
            placeholder='Coloque a pergunta aqui...',
            description='Pergunta:',
            layout=widgets.Layout(width='100%', height='80px')
        )

        self.answer_input = widgets.Textarea(
            placeholder='Escreva a resposta aqui... texto simples ou código HTML, gerado no LibGuides, por exemplo',
            description='Resposta:',
            layout=widgets.Layout(width='100%', height='150px')
        )

        self.add_button = widgets.Button(
            description='Adicionar Par P&R',
            button_style='success',
            icon='plus'
        )

        self.finish_button = widgets.Button(
            description='Concluir',
            button_style='danger',
            icon='check'
        )

        self.output_area = widgets.Output()
        self.preview_area = widgets.Output()

        # Set up event handlers
        self.add_button.on_click(self.add_qa_pair)
        self.finish_button.on_click(self.generate_code)

        # Display the UI
        display(widgets.HTML("<h2>Construtor de Acordeão LibGuides</h2>"))
        display(self.question_input)
        display(self.answer_input)
        display(widgets.HBox([self.add_button, self.finish_button]))
        display(widgets.HTML("<h3>Pares P&R Atuais:</h3>"))
        display(self.preview_area)
        display(widgets.HTML("<h3>Código Gerado:</h3>"))
        display(self.output_area)

    def add_qa_pair(self, button):
        question = self.question_input.value.strip()
        answer = self.answer_input.value.strip()

        if not question or not answer:
            with self.preview_area:
                clear_output()
                display(HTML("<div style='color: red'>Tanto a pergunta como a resposta são obrigatórias!</div>"))
                self.display_current_pairs()
            return

        self.qa_pairs.append((question, answer))
        self.question_input.value = ""
        self.answer_input.value = ""

        with self.preview_area:
            clear_output()
            display(HTML("<div style='color: green'>Par P&R adicionado com sucesso!</div>"))
            self.display_current_pairs()

    def display_current_pairs(self):
        if not self.qa_pairs:
            display(HTML("<p>Ainda não foram adicionados pares P&R.</p>"))
            return

        html = "<ol>"
        for q, a in self.qa_pairs:
            html += f"<li><strong>P: {q}</strong><br>R: {a[:100]}{'...' if len(a) > 100 else ''}</li>"
        html += "</ol>"

        display(HTML(html))

    def generate_code(self, button):
        if not self.qa_pairs:
            with self.output_area:
                clear_output()
                display(HTML("<div style='color: red'>Ainda não foram adicionados pares P&R!</div>"))
            return

        accordion_id = f"acordeao_{self.unique_id}"

        html_code = f"""
    <div class="custom-accordion" id="{accordion_id}">
    """

        for i, (question, answer) in enumerate(self.qa_pairs):
            item_id = f"item_{self.unique_id}_{i}"
            html_code += f"""
        <div class="accordion-item">
            <button class="accordion-trigger" id="{item_id}_trigger"
                    aria-expanded="false" aria-controls="{item_id}_content">
                <span class="accordion-title">{question}</span>
                <span class="accordion-icon">+</span>
            </button>
            <div class="accordion-content" id="{item_id}_content" hidden>
                {answer}
            </div>
        </div>
    """

        html_code += """
    </div>

    <style>
    .custom-accordion {
        max-width: 800px;
        margin: 1rem auto;
        font-family: Arial, sans-serif;
    }

    .accordion-item {
        border: 1px solid #ddd;
        margin-bottom: 0.5rem;
        border-radius: 4px;
    }

    .accordion-trigger {
        width: 100%;
        padding: 1rem;
        border: none;
        background: #f5f5f5;
        text-align: left;
        cursor: pointer;
        display: flex;
        justify-content: space-between;
        align-items: center;
    }

    .accordion-trigger:hover {
        background: #ebebeb;
    }

    .accordion-title {
        font-weight: bold;
        font-size: 1.1rem;
    }

    .accordion-icon {
        font-size: 1.5rem;
        line-height: 1;
    }

    .accordion-content {
        padding: 1rem;
        background: white;
    }

    .accordion-content[hidden] {
        display: none;
    }
    </style>

    <script>
    // Este script garante que apenas um item do acordeão está aberto de cada vez
    (function() {
        // Função para inicializar o acordeão
        function initAccordion() {
            const accordion = document.getElementById('""" + accordion_id + """');
            if (!accordion) return;

            const triggers = accordion.querySelectorAll('.accordion-trigger');

            triggers.forEach(trigger => {
                trigger.addEventListener('click', function() {
                    const isExpanded = this.getAttribute('aria-expanded') === 'true';
                    const contentId = this.getAttribute('aria-controls');
                    const content = document.getElementById(contentId);
                    const icon = this.querySelector('.accordion-icon');

                    // Primeiro, fechar todos os itens
                    triggers.forEach(otherTrigger => {
                        const otherContentId = otherTrigger.getAttribute('aria-controls');
                        const otherContent = document.getElementById(otherContentId);
                        const otherIcon = otherTrigger.querySelector('.accordion-icon');

                        otherTrigger.setAttribute('aria-expanded', 'false');
                        otherContent.hidden = true;
                        otherIcon.textContent = '+';
                    });

                    // Depois, abrir o item clicado se estava fechado
                    if (!isExpanded) {
                        this.setAttribute('aria-expanded', 'true');
                        content.hidden = false;
                        icon.textContent = '-';
                    }
                });
            });
        }

        // Executar inicialização quando o DOM estiver completamente carregado
        if (document.readyState === 'loading') {
            document.addEventListener('DOMContentLoaded', initAccordion);
        } else {
            initAccordion();
        }
    })();
    </script>
    """

        with self.output_area:
            clear_output()
            display(HTML("<div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'><p>Copie o código abaixo e cole-o no editor HTML/Texto Rico do LibGuides:</p></div>"))

            # Create a text area with the code for easy copying
            copy_textarea = widgets.Textarea(
                value=html_code,
                layout=widgets.Layout(width='100%', height='300px')
            )
            display(copy_textarea)

            # Add a copy button
            copy_button = widgets.Button(
                description='Copiar para a Área de Transferência',
                button_style='info',
                icon='copy'
            )

            def copy_to_clipboard(b):
                js_code = f"""
                const textarea = document.querySelector('textarea');
                textarea.select();
                document.execCommand('copy');
                """
                display(HTML(f"<script>{js_code}</script>"))
                display(HTML("<div style='color: green; margin-top: 10px;'>Código copiado para a área de transferência!</div>"))

            copy_button.on_click(copy_to_clipboard)
            display(copy_button)

            # Preview
            display(HTML("<h4>Pré-visualização:</h4>"))
            display(HTML(html_code))

# Run the builder
builder = LibGuidesAccordionBuilder()

HTML(value='<h2>Construtor de Acordeão LibGuides</h2>')

Textarea(value='', description='Pergunta:', layout=Layout(height='80px', width='100%'), placeholder='Coloque a…

Textarea(value='', description='Resposta:', layout=Layout(height='150px', width='100%'), placeholder='Escreva …

HTML(value='<h3>Pares P&R Atuais:</h3>')

Output()

HTML(value='<h3>Código Gerado:</h3>')

Output()